# Dask with cuDF and XGBoost from Disk
#### By Paul Hendricks
-------

While the world’s data doubles each year, CPU computing has hit a brick wall with the end of Moore’s law. For the same reasons, scientific computing and deep learning has turned to NVIDIA GPU acceleration, data analytics and machine learning where GPU acceleration is ideal. 

NVIDIA created RAPIDS – an open-source data analytics and machine learning acceleration platform that leverages GPUs to accelerate computations. RAPIDS is based on Python, has pandas-like and Scikit-Learn-like interfaces, is built on Apache Arrow in-memory data format, and can scale from 1 to multi-GPU to multi-nodes. RAPIDS integrates easily into the world’s most popular data science Python-based workflows. RAPIDS accelerates data science end-to-end – from data prep, to machine learning, to deep learning. And through Arrow, Spark users can easily move data into the RAPIDS platform for acceleration.

In this notebook, we will show how to quickly setup Dask and train an XGBoost model using cuDF and read the data from disk using cuIO.

**Table of Contents**

* Setup
* Load Libraries
* Setup Dask
* Generate Data and Write to Disk
* Use cuIO to Load GPU DataFrames using Dask
* Train Model
* Conclusion

## Setup

This notebook was tested using the `nvcr.io/nvidia/rapidsai/rapidsai:0.5-cuda10.0-runtime-ubuntu18.04-gcc7-py3.7` Docker container from [NVIDIA GPU Cloud](https://ngc.nvidia.com) and run on the NVIDIA Tesla V100 GPU. Please be aware that your system may be different and you may need to modify the code or install packages to run the below examples. 

If you think you have found a bug or an error, please file an issue here: https://github.com/rapidsai/notebooks/issues

Before we begin, let's check out our hardware setup by running the `nvidia-smi` command.

In [ ]:
!nvidia-smi

Next, let's see what CUDA version we have:

In [ ]:
!nvcc --version

## Load libraries

In [ ]:
import cudf
import dask.dataframe as dd
from dask.delayed import delayed
from dask.distributed import Client, wait
import dask_xgboost as dxgb_gpu
import numpy as np
import os
import pandas as pd
from pandas.util.testing import assert_frame_equal
import subprocess
import time
import xgboost as xgb

## Setup Dask

Dask is a library the allows for parallelized computing. Written in Python, it allows one to schedule tasks and do so dynamically as well handle large data structures - similar to those found in NumPy and Pandas. In the subsequent tutorials, we'll show how to use Dask with Pandas and cuDF and how we can use both to accelerate common ETL tasks as well as build ML models like XGBoost.

To learn more about Dask, check out the documentation here: http://docs.dask.org/en/latest/

Dask operates using a concept of a "Client" and "workers". The client tells the workers what tasks to perform and when to perform. Typically, we set the number of works to be equal to the number of computing resources we have available to us. For example, wer might set `n_workers = 8` if we have 8 CPU cores on our machine that can each operate in parallel. This allows us to take advantage of all of our computing resources and enjoy the most benefits from parallelization.

Dask is a first class citizen in the world of General Purpose GPU computing and the RAPIDS ecosystem makes it very easy to use Dask with cuDF and XGBoost. As we see below, we can inititate a Cluster and Client using only few lines of code.

In [ ]:
from dask_cuda import LocalCUDACluster

cluster = LocalCUDACluster()
client = Client(cluster)

Now, let's show our current Dask status. We should see the IP Address for our Scheduler as well the the number of workers in our Cluster. 

In [ ]:
# show current Dask status
client

You can also see the status and more information at the Dashboard, found at `http://<scheduler_uri>/status`. You can ignore this for now, we'll dive into this in subsequent tutorials.

## Generate Data and Write to Disk

In [ ]:
def make_pandas(n_rows, n_columns):
    X = np.random.randint(2, size=(n_rows, n_columns)).astype(np.float32)
    df = pd.DataFrame({'fea%d'%i:X[:,i] for i in range(X.shape[1])})
    return df

In [ ]:
# dataset_sizes = [(10000, 100)]
dataset_size = (12500, 101)  # foo
print('Final dataset size: ({}, {})'.format(dataset_size[0] * 8, dataset_size[1]))

# base_path = '/datasets/rapids/dask-examples/datasets'
base_path = '/tmp/datasets'
if not os.path.isdir(base_path):
    os.mkdir(base_path)

In [ ]:
n_workers = 8
for i in range(n_workers):
    filename = 'dataset-{}.csv'.format(i)
    filepath = os.path.join(base_path, filename)
    n_rows, n_columns = dataset_size
    df = make_pandas(n_rows, n_columns)
    print('Writing:', filename)
    df.to_csv(filepath, index=False)
    print('Finished!')

In [ ]:
!ls -alh /tmp/datasets

## Use cuIO to Load GPU DataFrames using Dask

In [ ]:
def initialize_rmm_pool():
    from librmm_cffi import librmm_config as rmm_cfg

    rmm_cfg.use_pool_allocator = True
    #rmm_cfg.initial_pool_size = 2<<30 # set to 2GiB. Default is 1/2 total GPU memory
    import cudf
    return cudf._gdf.rmm_initialize()

def initialize_rmm_no_pool():
    from librmm_cffi import librmm_config as rmm_cfg
    
    rmm_cfg.use_pool_allocator = False
    import cudf
    return cudf._gdf.rmm_initialize()

In [ ]:
client.run(initialize_rmm_pool)

In [ ]:
def read_csv(filepath):
    n_columns = 101
    dtypes = ['float'] * n_columns
    names = ['fea{}'.format(i) for i in range(n_columns)]
    gdf = cudf.io.csv.read_csv(filepath, names=names, dtype=dtypes, skiprows=1)
    return gdf


def make_features(gdf):
    column_names = gdf.columns
    return gdf.loc[:, column_names[1:]]


def make_labels(gdf):
    column_names = gdf.columns
    return gdf.loc[:, column_names[:1]]

In [ ]:
base_path = '/tmp/datasets'
filepaths = [os.path.join(base_path, 'dataset-{}.csv'.format(i)) for i in range(n_workers)]

In [ ]:
gdfs = [delayed(read_csv)(filepath) for filepath in filepaths]

In [ ]:
client.run(cudf._gdf.rmm_finalize)

In [ ]:
client.run(initialize_rmm_no_pool)

In [ ]:
gpu_dfs = [[delayed(make_features)(gdf), delayed(make_labels)(gdf)] for gdf in gdfs]

In [ ]:
gpu_dfs = [delayed(xgb.DMatrix)(gpu_df[0], gpu_df[1]) for gpu_df in gpu_dfs]
gpu_dfs = [gpu_df.persist() for gpu_df in gpu_dfs]

In [ ]:
wait(gpu_dfs)

## Train Model

In [ ]:
dxgb_gpu_params = {
    'nround':            1000,
    'max_depth':         8,
    'max_leaves':        2**8,
    'alpha':             0.9,
    'eta':               0.1,
    'gamma':             0.1,
    'learning_rate':     0.1,
    'subsample':         1,
    'reg_lambda':        1,
    'scale_pos_weight':  2,
    'min_child_weight':  30,
    'tree_method':       'gpu_hist',
    'n_gpus':            1,
    'distributed_dask':  True,
    'loss':              'ls',
    'objective':         'gpu:reg:linear',
    'max_features':      'auto',
    'criterion':         'friedman_mse',
    'grow_policy':       'lossguide',
    'verbose':           True
}

In [ ]:
%%time
labels = None
bst = dxgb_gpu.train(client, dxgb_gpu_params, gpu_dfs, labels, num_boost_round=dxgb_gpu_params['nround'])

In [ ]:
bst

## Conclusion

To learn more about RAPIDS, be sure to check out: 

* [Open Source Website](http://rapids.ai)
* [GitHub](https://github.com/rapidsai/)
* [Press Release](https://nvidianews.nvidia.com/news/nvidia-introduces-rapids-open-source-gpu-acceleration-platform-for-large-scale-data-analytics-and-machine-learning)
* [NVIDIA Blog](https://blogs.nvidia.com/blog/2018/10/10/rapids-data-science-open-source-community/)
* [Developer Blog](https://devblogs.nvidia.com/gpu-accelerated-analytics-rapids/)
* [NVIDIA Data Science Webpage](https://www.nvidia.com/en-us/deep-learning-ai/solutions/data-science/)
